<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/%20Google%20Earth%20Engine%20with%20Amirhossein%20Ahrari/001_lake_area_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap
import ee
from datetime import datetime, timedelta

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

In [2]:
def filterImageCollection(
        image_collection_id: str,
        geometry: ee.geometry.Geometry,
        date_range: tuple[str],
        cloud_cover = 10
    ) -> ee.imagecollection.ImageCollection:

    imageCollection = ee.ImageCollection(image_collection_id)
    d1,d2 = date_range
    filtered = imageCollection.filter(
       ee.Filter.lt('CLOUD_COVER', 10)
    )\
    .filter(
        ee.Filter.date(d1, d2) ## Filter by date
    ).filter(
        ee.Filter.bounds(geometry) ## Filter by bounds
    )

    return filtered

In [3]:
## get the current dates
today = datetime.now()
d1 = today - timedelta(days=180)
current_date_range = (d1.date().strftime('%Y-%m-%d'), today.date().strftime('%Y-%m-%d'))
current_date_range

('2023-10-28', '2024-04-25')

## Landsat data


**What are some major differences between Landsat 8 and Landsat 9 ?**

In summary, while Landsat 8 and Landsat 9 share a similar mission and purpose, Landsat 9 represents an advancement in technology and continuity for the Landsat program. It provides improved sensor capabilities and reliability, ensuring the continuation of high-quality Earth observation data critical for scientific research and environmental management.

**Which product to choose from Tier 1 or Tier 2 ?**

In summary, Landsat 9 Tier 1 data products offer superior quality and precision suitable for scientific and research applications, while Tier 2 data is more accessible and adequate for general mapping and visualization tasks. Choose based on your specific project requirements and the level of precision needed for your analysis

In [4]:
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
delhi = admin.filter(ee.Filter.eq('ADM1_NAME', 'Delhi'))\
.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))

geometry = delhi.geometry()

filtered_imgs = filterImageCollection(
    image_collection_id = "LANDSAT/LC08/C02/T1_L2",
    geometry = geometry,
    date_range = current_date_range
)

filtered_imgs.size()

Now process the lansat images

- scale images
- calculate ndwi

In [5]:
def preprocess_landsat(image:ee.image.Image) -> ee.image.Image:
    '''
    This function will choose only the required bands
    also rescale the reflectance value to original
    and calculate the indices: NDVI
    '''

    ## Select and rescale
    bands = image.select("SR_*").multiply(2.75e-05).add(-2)

    ## Calculate the NDWI
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')

    return ndwi.copyProperties(image, image.propertyNames())


landsat = filtered_imgs.map(preprocess_landsat)

## Plot the NDWI data

In [6]:
Map = geemap.Map()

## Calculate the median composite and select values greater than 0 only
ndwi = landsat.median().clip(geometry)

## Mask the image: this will set all the values
thr = ndwi.gt(0)
ndwi_masked = thr.updateMask(thr)

vizParams = {
    'min': 0,
    'max': 1,
    'palette': ['#ece7f2','#a6bddb','#2b8cbe']
}

Map.addLayer(ndwi_masked.clip(geometry), vizParams, 'NDWI')

Map.centerObject(geometry, 12)
Map

Map(center=[28.64654955564474, 77.10895454636062], controls=(WidgetControl(options=['position', 'transparent_b…

Masking the NDWI will set value 1 for pixels with ndwi more than 0 and rest will be set to 0


## Calculating the area

Using the [`ee.Image.pixelArea()`](https://developers.google.com/earth-engine/apidocs/ee-image-pixelarea): This will calculate the area of each pixel of the image

In [ ]:
pixel_area = ndwi_masked.multiply(ee.Image.pixelArea().divide(1e6)) ## Area in km2

Now calculate the total area of the masked image using `reducers`

In [17]:
water_area = pixel_area.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = geometry,
    scale = 30
)

water_area = water_area.getInfo()['ndwi']
print(f"The total area of water bodies in Delhi is {water_area} Km2")

The total area of water bodies in Delhi is 7.817107440852668 Km2
